In [1]:
import pandas as pd
import os

In [2]:
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [3]:
def calCulateBrick(base_area):
    bricks=round((base_area*32)/6,4)
    return bricks
def calculateConcreteMaterial(cem,ca,fa,block_vol):
    cem_w=cem*1440
    sand_w=ca*1600
    stone_w=fa*1700    
    sand_r=sand_w/cem_w
    stone_r=stone_w/cem_w
    water_r=0.45
    print("{}:{}:{}".format(1,sand_r, stone_r))
    cem_vol=1/(3150.0)
    sand_vol= sand_r/(2600.0)
    stone_vol=stone_r/(2700.0)
    water_vol=water_r/(1000.0)    
    un_compacted_volume_perkg=cem_vol+sand_vol+stone_vol+water_vol
    compacted_volume_perkg=un_compacted_volume_perkg*0.97
    print("un_compact_vol={} compact_vol={}".format(un_compacted_volume_perkg,compacted_volume_perkg))
    cement_in_weight=block_vol/compacted_volume_perkg
    sand_in_weight=cement_in_weight*sand_r
    stone_in_weight=cement_in_weight*stone_r
    print("in kg cement={} sand={} stone={}".format(cement_in_weight,sand_in_weight,stone_in_weight))
    sand_in_vol=sand_in_weight/1600.0
    stone_in_vol= stone_in_weight/1550.0
    myvalue={'cement':cement_in_weight,'sand':sand_in_vol,'stone':stone_in_vol}
    print(myvalue)
    return myvalue    
    
    
    

In [4]:
def calcualteFormWork(perimeter,height):
    h=height
    plain_sheet=round(perimeter*height,4)
    angle_length=2*perimeter+6*h
    angle1=(2*perimeter+6*h)*1.9157
    angle1=round(angle1,4)
    angle2=angle1/5855
    angle2=round(angle2,4)
    welding=int(angle_length*24)
    myvalue={'angle1':angle1,'angle2':angle2,'plain_sheet':plain_sheet,'welding':welding}
    print(myvalue)
    return myvalue

In [5]:
def calcualteQuantity(template,input_df,prod_data,cost_data):
    mylist=list(template['rate'])
    for index,row in input_df.iterrows():
        vat_rate=0.075
        code=row['codeNo']
        mydata=[]
        mycost_data=[]
        L=row['Length']/100
        W=row['Bbreadth']/100
        H=row['Height']/100
        volume=L*W*H
        perimeter=2*(L+H)
        base_area=L*H
        print("volume={} perimeter={} base_area={}".format(volume,perimeter,base_area))
        """adding brick"""
        brick=calCulateBrick(base_area)
        
        "adding bricks"
        mydata.append(brick)
        mycost_data.append(brick*template.iloc[0,3])
       
        print("brick={}".format(brick))
        "adding sand,cement,stone"
        cem=row['Cement']
        sand=row['Sand']
        stone=row['Stone']
        myvalue=calculateConcreteMaterial(cem,sand,stone,volume)
        mydata.append(myvalue['cement'])
        mydata.append(myvalue['sand'])
        mydata.append(myvalue['stone'])
        mycost_data.append(myvalue['cement']*template.iloc[1,3])
        mycost_data.append(myvalue['sand']*template.iloc[2,3])
        mycost_data.append(myvalue['stone']*template.iloc[3,3])
        "adding disel and mobil"
        disel=prod_data.iloc[0,3]*volume
        mobil=prod_data.iloc[1,3]*volume
        print("disel={} mobil={}".format(disel,mobil))
        mydata.append(disel)
        mydata.append(mobil)
        mycost_data.append(disel*template.iloc[4,3])
        mycost_data.append(mobil*template.iloc[5,3])
        polythin=base_area*1.5
        """adding polythene"""
        mydata.append(polythin)
        mycost_data.append(polythin*template.iloc[6,3])
        print("polythin={}".format(polythin))
        
        putty=round(prod_data.iloc[2,3]*volume,4)
        print("putty={}".format(putty))
        """adding putty"""
        mydata.append(putty)
        mycost_data.append(putty*template.iloc[7,3])
        """Adding Metal Works"""
        formwork=calcualteFormWork(perimeter,H)
        mydata.append(formwork['angle1'])
        mydata.append(formwork['angle2'])
        mydata.append(formwork['plain_sheet'])
        mydata.append(formwork['welding'])
        mycost_data.append(formwork['angle1']*template.iloc[8,3]*0.015)
        mycost_data.append(formwork['angle2']*template.iloc[9,3]*0.015)
        mycost_data.append(formwork['plain_sheet']*template.iloc[10,3]*0.015)
        mycost_data.append(formwork['welding']*template.iloc[11,3]*0.015)
        """Adding Equipments"""
        mixture_machine=round(prod_data.iloc[3,3]*volume,4)
        mydata.append(mixture_machine)
        mycost_data.append(mixture_machine*template.iloc[12,3])
        """Adding Labours"""
        unskilled=round(prod_data.iloc[4,3]*volume,4)
        masson=round(prod_data.iloc[5,3]*volume,4)
        headmasson=round(prod_data.iloc[6,3]*volume,4)
        messonhelper=round(prod_data.iloc[7,3]*volume,4)
        operator=round(prod_data.iloc[8,3]*volume,4)
        welder=round(prod_data.iloc[9,3]*formwork['welding'],4)
        print("unsk={} m={} mhd={} mhl={} op={} welder={}".format(unskilled,masson,
                                                                  headmasson,messonhelper,operator,welder))
        mydata.append(unskilled)       
        mydata.append(masson)
        mydata.append(headmasson)
        mydata.append(messonhelper)
        mydata.append(operator)
        mydata.append(welder)
        
        mycost_data.append(unskilled*template.iloc[13,3])
        mycost_data.append(masson*template.iloc[14,3])        
        mycost_data.append(headmasson*template.iloc[15,3])
        mycost_data.append(messonhelper*template.iloc[16,3])
        mycost_data.append(operator*template.iloc[17,3])
        mycost_data.append(welder*template.iloc[18,3])     
        "trasfering analysis data to data frame"
        template[code]=mydata
        """Calcualting overhead,vat,tax"""
        sum_sub_total=sum(mycost_data)
        mycost_data.append(sum_sub_total)
        overhead=0.025*sum_sub_total
        mycost_data.append(overhead)
        item_total= sum_sub_total+overhead
        mycost_data.append(item_total)
        cp=item_total*0.1
        mycost_data.append(cp)
        total=item_total+cp
        mycost_data.append(total)
        vat=vat_rate*total
        mycost_data.append(vat)
        grand_total=total+vat
        mycost_data.append(grand_total)
        print(template.info())
        print(mydata)
        print(mycost_data)
        cost_data[code]=mycost_data
        myvalues={'q':template,'c':cost_data}
    return myvalues

In [6]:
myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Analysis Template'
input_path=os.path.join(myfolder,'AnalysisInput.xlsx')
output_path=os.path.join(myfolder,'analysis_dframe.xlsx')

In [7]:
sheetName='Template_Quantity'
q_template_df=pd.read_excel(input_path,sheet_name=sheetName)
sheetName='Production Rate'
production_rate_df=pd.read_excel(input_path,sheet_name=sheetName)
production_rate_df
sheetName='Block Data'
block_data_df=pd.read_excel(input_path,sheet_name=sheetName)
block_data_df
sheetName='Template_Cost'
c_template_df=pd.read_excel(input_path,sheet_name=sheetName)
"""Analysing data"""
myvalues=calcualteQuantity(q_template_df,block_data_df,production_rate_df,c_template_df)
q_template_df=myvalues['q']
c_template_df=myvalues['c']



volume=0.108 perimeter=1.7999999999999998 base_area=0.18
brick=0.96
1:3.3333333333333335:7.083333333333333
un_compact_vol=0.004672968389635056 compact_vol=0.004532779337946004
in kg cement=23.82644111878153 sand=79.42147039593844 stone=168.77062459136917
{'cement': 23.82644111878153, 'sand': 0.04963841899746153, 'stone': 0.10888427392991559}
disel=0.1525284 mobil=0.0076464
polythin=0.27
putty=0.0036
{'angle1': 10.3448, 'angle2': 0.0018, 'plain_sheet': 0.54, 'welding': 129}
unsk=0.3091 m=0.0201 mhd=0.0037 mhl=0.0094 op=0.0064 welder=0.0022
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codeno       19 non-null     object 
 1   Description  19 non-null     object 
 2   Unit         19 non-null     object 
 3   rate         19 non-null     float64
 4   40-150-03    19 non-null     float64
dtypes: float64(2), object(3)
memory usage: 888.0+ bytes
No

In [8]:
myframes=[]
mynames=[]
myframes.append(q_template_df)
mynames.append('template')
myframes.append(production_rate_df)
mynames.append('production_rate')
myframes.append(block_data_df)
mynames.append('analysis_list')
myframes.append(c_template_df)
mynames.append('Analyzed_cost')
saveDataFrame(myframes,output_path,mynames)

In [9]:
production_rate_df

,code,Description,unit,consumption/unit volume,Production /unit
0,20-245,Diesel/ Fuel,litr/cum,1.412300,0.708100
1,20-310,Lubricant ( Mobile oil etc.),litr/cum,0.070800,14.124300
2,20-505,Putty,kg/cum,0.032900,30.395137
3,70-080,Mixture Machine 0.2 cum,day/cum,0.059300,16.863406
4,80-375,"Labour,Unskilled",day/cum,2.862000,0.349406
5,80-425,Mason,day/cum,0.186000,5.376344
6,80-430,"Mason, Head",day/cum,0.034600,28.901734
7,80-435,"Mason, Helper",day/cum,0.087200,11.467890
8,80-500,"Operator, Mixture Machine",day/cum,0.059300,16.863406
9,80-755,Welder,day/point,0.000017,59545.454545


In [10]:
c_template_df

,codeno,Description,Unit,rate,40-150-03,40-170-03,40-190-03,40-210-03
0,10-040,Bricks : 1st Class,Nos,9.5,9.120000,9.120000,9.120000,9.120000
1,10-100,Cement : Portland .,Kgs,10.0,238.264411,250.236405,276.824494,330.275392
2,10-380,"Sand, FM >= 1.5",cum,1050.0,52.120340,63.862416,50.462798,48.165161
3,10-475,"Stone, Chips : 40mm down graded",cum,6500.0,707.747781,619.424793,685.239844,654.039978
4,20-245,Diesel/ Fuel,litre,65.0,9.914346,9.914346,9.914346,9.914346
5,20-310,Lubricant ( Mobile oil etc.),litre,400.0,3.058560,3.058560,3.058560,3.058560
6,20-500,Polythene Sheet : (1.0 kg per 16.00 sqm),sqm,15.0,4.050000,4.050000,4.050000,4.050000
7,20-505,Putty,kg,80.0,0.288000,0.288000,0.288000,0.288000
8,40-020,"Angle, M.S",kg,60.0,9.310320,9.310320,9.310320,9.310320
9,40-020,"(Angle, M.S. .... ... ... ... ... ... ...)",kg,60.0,0.001620,0.001620,0.001620,0.001620
